 ## Load the cleaned dataset

# global imports


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib 

In [8]:


df = pd.read_csv("data/cleaned_amazon_reviews_final.csv")
print(f"Loaded dataset shape: {df.shape}")
print(df.head)
df.columns


Loaded dataset shape: (4385, 3)
<bound method NDFrame.head of                                              clean_text  \
0     thought would big small paper turn like palm t...   
1                     kindl light easi use especi beach   
2     didnt know much id use kindl went lower end im...   
3     happi purchas caught sale realli good price no...   
4     solid entri level kindl great kid gift kid fri...   
...                                                 ...   
4380  yr old son love tableti plan purchas anoth fir...   
4381  appreci eas purchas read found everi book titl...   
4382                 great tablet price amazon good job   
4383  tablet perfect size easi use read play game pu...   
4384  thought get year old get screen protector case...   

                                             categories  label  
0     Computers,Electronics Features,Tablets,Electro...      1  
1     Computers,Electronics Features,Tablets,Electro...      2  
2     Computers,Electronics Featur

Index(['clean_text', 'categories', 'label'], dtype='object')

# Prepare the Data

In [9]:
X = df['review_text']
y = df['sentiment']

KeyError: 'review_text'